In [1]:
import glob
import os
import numpy as np
import math
from DataProcess import *
from SolutionChecker import *
from LocalSearch import *
import cProfile

In [2]:
data_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"

In [3]:
solution_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.solution"

In [4]:
print_data(data_path)

Add evacuation info
Add graph info
-----------------------------
evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  5
-----------------------------


0

In [5]:
#print_solution(solution_path)

In [6]:
#verify_solution(data_path,solution_path)

In [7]:
EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

Add evacuation info
Add graph info


In [8]:
# for i in range(NB_EVA_NODES) : 
#     node_id = EVA_TREE[i][0]
#     print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id,EVA_TREE,GRAPH),' mins')

In [9]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 34 mins


In [10]:
endtime,init_sol= get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)
print('Borne superieur = {} mins'.format(endtime))
init_sol

Borne superieur = 40 mins


[[1, 8, 0], [3, 3, 0], [2, 5, 9]]

In [11]:
data_path_2 = os.path.dirname(os.path.abspath('__file__')) + "/data/dense.data"
EVA_TREE,GRAPH,NB_NODES = read_data(data_path_2)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

Add evacuation info
Add graph info


In [12]:
#print_data(data_path_2)

In [13]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 91 mins


In [30]:
endtime,init_sol= get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)
print('Borne superieur = {} mins'.format(endtime))
init_sol

Borne superieur = 196 mins


[[268, 132, 0],
 [359, 70, 0],
 [279, 71, 3],
 [275, 70, 21],
 [323, 71, 45],
 [285, 71, 63],
 [384, 71, 82],
 [163, 70, 60],
 [227, 71, 126],
 [344, 71, 150]]

In [31]:
cp = cProfile.Profile()
cp.enable()
endtime,best_solution= LocalSearchRun(init_sol,EVA_TREE,GRAPH,n_iter=10)
cp.disable()
cp.print_stats()

Iteration 0:
[268, 359, 279, 275, 323, 163, 285, 384, 227, 344]  =>  196
Iteration 1:
[285 359 279 275 323 163 268 384 227 344]  =>  180
Iteration 2:
[285 359 279 275 323 163 268 384 227 344]  =>  180
Iteration 3:
[285 359 279 275 323 163 268 384 227 344]  =>  180
Iteration 4:
[285 359 279 275 323 163 268 384 227 344]  =>  180
Iteration 5:
[285 359 279 275 323 163 268 384 227 344]  =>  180
         5610110 function calls (5610060 primitive calls) in 8.312 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    8.312    8.312 <ipython-input-31-384d44187790>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-31-384d44187790>:4(<module>)
     2710    0.015    0.000    1.195    0.000 DataProcess.py:102(get_task)
     2710    0.023    0.000    1.128    0.000 DataProcess.py:106(<listcomp>)
      271    0.002    0.000    0.515    0.002 DataProcess.py:121(create_solution)
     2710    0.

In [32]:
cp = cProfile.Profile()
cp.enable()
endtime,best_solution= LocalSearchRun2(init_sol,EVA_TREE,GRAPH,n_iter=10)
cp.disable()
cp.print_stats()

Iteration 0:
[268, 359, 279, 275, 323, 163, 285, 384, 227, 344]  =>  196
Iteration 1:
[268 227 279 275 323 163 285 384 359 344]  =>  181
Iteration 2:
[268 227 279 275 323 163 285 384 359 344]  =>  181
Iteration 3:
[268 227 279 275 323 163 285 384 359 344]  =>  181
Iteration 4:
[268 227 279 275 323 163 285 384 359 344]  =>  181
Iteration 5:
[268 227 279 275 323 163 285 384 359 344]  =>  181
         2741141 function calls (2741091 primitive calls) in 4.353 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.353    4.353 <ipython-input-32-368d4457975d>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-32-368d4457975d>:4(<module>)
     2470    0.009    0.000    1.027    0.000 DataProcess.py:102(get_task)
     2470    0.020    0.000    0.977    0.000 DataProcess.py:106(<listcomp>)
      127    0.001    0.000    0.236    0.002 DataProcess.py:121(create_solution)
     1270    0.

In [17]:
# cp = cProfile.Profile()
# cp.enable()
# best_solution,endtime = LocalSearchRun2(init_sol,EVA_TREE,GRAPH,n_iter=10)
# cp.disable()
# cp.print_stats()

In [33]:
print('solution = ',best_solution)
print('end time = ',endtime)

solution =  [[268, 132, 0], [227, 71, 0], [279, 71, 6], [275, 70, 21], [323, 71, 45], [163, 70, 60], [285, 71, 66], [384, 71, 85], [359, 70, 114], [344, 71, 135]]
end time =  181


In [19]:
cp = cProfile.Profile()
cp.enable()
endtime,best_solution = LocalSearchRandomStart(EVA_TREE,GRAPH,n_iter=10,n_start_points=2)
cp.disable()
cp.print_stats()
print('solution = ',best_solution)
print('end time = ',endtime)

---------------------------Start n.1---------------------------
Iteration 0:
[279, 285, 163, 344, 359, 323, 275, 227, 268, 384]  =>  184
Iteration 1:
[323 285 163 344 359 279 275 227 268 384]  =>  180
Iteration 2:
[323 285 163 344 359 279 275 227 268 384]  =>  180
Iteration 3:
[323 285 163 344 359 279 275 227 268 384]  =>  180
Iteration 4:
[323 285 163 344 359 279 275 227 268 384]  =>  180
Iteration 5:
[323 285 163 344 359 279 275 227 268 384]  =>  180
----------------------------------------------------------------
---------------------------Start n.2---------------------------
Iteration 0:
[285, 163, 359, 344, 384, 275, 279, 323, 268, 227]  =>  180
Iteration 1:
[285, 163, 359, 344, 384, 275, 279, 323, 268, 227]  =>  180
Iteration 2:
[285, 163, 359, 344, 384, 275, 279, 323, 268, 227]  =>  180
Iteration 3:
[285, 163, 359, 344, 384, 275, 279, 323, 268, 227]  =>  180
Iteration 4:
[285, 163, 359, 344, 384, 275, 279, 323, 268, 227]  =>  180
-------------------------------------------------

In [20]:
create_solution_file('data/wildfire_sup.solution',best_solution,endtime)

0

In [21]:
# #-----------------------------------------------------------------------------
# # Build the model
# #-----------------------------------------------------------------------------
# from docplex.cp.model import CpoModel
# from docplex.cp.expression import *
# from docplex.cp.solution import *
# import docplex.cp.utils_visu as visu
# # Create model
# mdl = CpoModel(version='12.7.0')

In [22]:
# # Create one interval variable per job operation
# E_nodes = [[mdl.interval_var(size=get_task(j,EVA_TREE,GRAPH)[0], name="Evacuees from {} at {}".format(j,m)) for m in get_task(j,EVA_TREE,GRAPH)[1]] for j in LIST_EVA_NODES]
# len(E_nodes)

In [23]:
# len(E_nodes[0])

In [24]:
# #-----------------------------------------------------------------------------
# # Constraints 
# #-----------------------------------------------------------------------------

# #Each operation must start after a precise duration = time to traverse an arc
# for i in range(NB_EVA_NODES) :
#     root = LIST_EVA_NODES[i]
#     route_list = get_task(root,EVA_TREE,GRAPH)[1]
#     route_len = len(route_list)
#     current = root
#     for m in range(route_len) :
#         nxt = route_list[m]
# #         print("Node-{}-to-{}".format(root,nxt))
#         if nxt != current :
# #             print(current,nxt)
#             _,delay,_ = get_edge_info(current,nxt,GRAPH)
#             mdl.add(mdl.start_at_start(E_nodes[i][m-1],E_nodes[i][m],delay))
#             print("Evacuees-from-{}-at-{} starts after Evacuees-from-{}-at-{} : {} mins ".format(root,nxt,root,current,delay))
#         current = nxt
#     print('----------')

In [25]:
# #Constraint on capacity of ressources, which are edges of graph

# for r in range(NB_EDGES) :
#     edge = GRAPH[r]
# #     print(edge)
#     CAP_MAX = edge[-1]
#     rsc = 0
#     ressources = []
#     for i in range(NB_EVA_NODES) :
#         root = LIST_EVA_NODES[i]
#         _,route_list,capacity = get_task(root,EVA_TREE,GRAPH)
#         route_len = len(route_list)
#         current = root

#         for m in range(route_len) : 
#             nxt = route_list[m]

#             if edge[0] == current and edge[1]==nxt :
#                 ressources.append(mdl.pulse(E_nodes[i][m-1],capacity))
#                 print("Evacuees-from-{}-at-{} requests edge [{}-{}] a capacity {}".format(root,route_list[m-1],current,nxt,capacity))
                
#                 rsc += capacity
               
#             current = nxt
#         #end for
#     #end for
#     mdl.add(mdl.sum(ressources) <= CAP_MAX)
#     print('Constraint on edge {}-{} with capacity_max {}... is loaded'.format(edge[0],edge[1],CAP_MAX))
#     print('--------------')
# #end for

In [26]:
# #Minimize end of all tasks
# mdl.add(mdl.minimize(mdl.max([mdl.end_of(node) for list_nodes in E_nodes for node in list_nodes])))

In [27]:
# #-----------------------------------------------------------------------------
# # Solve the model and display the result
# #-----------------------------------------------------------------------------
# mdl.write_information()

In [28]:
# # Solve model
# # print("Solving model....")
# msol = mdl.solve(TimeLimit=1000)
# # print("Solution: ")
# msol.print_solution()